In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.integrate import solve_ivp,odeint,ode
import seaborn as sns
import pandas as pd
#from multiprocessing import Pool
#import multiprocessing as mp

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Adding M compartments and changing initial values accordingly

In [3]:
### Defining Parameters

# Number of compartments ( > 1)
M = 3

#Time parameters of simulation

t_max = 1000*24*60 #maximum (mins)
dt = 60  #step (mins)
t_steps = t_max / dt

#Initial Values

##cells
y0_Tpos = np.ones(M) * 1000
y0_Tpro = np.ones(M) * 1000
y0_Tneg = np.ones(M) * 1000

##resource
y0_o2 = np.ones(M) * 0.5 #(prop)
y0_test= np.zeros(M) #(prop)


#Production rates of resources

p_o2 = np.ones(M) * 0.11 #(prop/min)
p_test = 5E-7 #by Tp cells (prop/min/cell)


#Uptake rate of resources by cells

##oxygen
mu_o2Tpos = 1.63E-6 #(prop/min/cell)
mu_o2Tpro = 1.63E-6 #(prop/min/cell)
mu_o2Tneg = 1.04E-6 #(prop/min/cell)

##testosterone
mu_testTpos = 2.34E-8 #(prop/min/cell)
mu_testTpro = 6E-8 #(prop/min/cell)

#Decay rates of resources
lam_o2 = 0.1 #(1/min)
lam_test = 0.004 #(1/min)

#Doubling time
t_DTpos = 34*60 #(min)
t_DTpro = 40*60 #(min)
t_DTneg = 25*60 #(min)

#Growth rate
r_Tpos = 2.84E-3 #(/min)
r_Tpro = 2.79E-3 #(/min)
r_Tneg = 6.23E-4 #(/min)

#Death rate
delta_Tpos = 2.5E-3 #(/min)
delta_Tpro = 2.5E-3 #(/min)
delta_Tneg = 1.6E-4 #(/min)

#Min Carrying capacity
K = 1

#Environmental Carrying capacity (Scaling Factor)
rho_Tpos = 8.35E4
rho_Tpro = 9.62E4
rho_Tneg = 1.34E4

# Maximum Capacity per compartment
K_Tpos = (K + rho_Tpos)/M
K_Tpro = (K + rho_Tpro)/M
K_Tneg = (K + rho_Tneg)/M

#Resource limits

##Oxygen
###T+
l_lim_o2Tpos = 0 #lower-threshold(prop)
u_lim_o2Tpos = 1.1 #upper-saturation(prop)

###Tp
l_lim_o2Tpro = 0 #(prop)
u_lim_o2Tpro = 1.1 #(prop)

###T-
l_lim_o2Tneg = 0 #(prop)
u_lim_o2Tneg = 1.1 #(prop)

##Testosterone
###T+
l_lim_testTpos = 0 #(prop)
u_lim_testTpos = 0.1 #(prop)

###Tp
l_lim_testTpro = 0 #(prop)
u_lim_testTpro = 0.1 #(prop)

###Diffusion Coefficients

# https://www.nature.com/articles/2151173a0 - 3.6E-5 in rat liver
# https://pubmed.ncbi.nlm.nih.gov/563582/ - 1.75E-5 in rat carcinoma
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1204697/ - 1.64E-5 in rat liver

D_o2 = 60 * 1.7E-5

# https://www.sciencedirect.com/science/article/abs/pii/S0022354915503316 - 1.9E-6 in pig intestinal mucous
D_test = 60 * 1.9E-6


In [4]:
### Creating initial values dataset

#parsing input into numpy arrays

y0 = []
for i in range(M):
    y0.append(y0_Tpos[i])
    y0.append(y0_Tpro[i])
    y0.append(y0_Tneg[i])
    y0.append(y0_o2[i])
    y0.append(y0_test[i])

    #change p to list, int type
p = [p_o2, p_test]

mu = np.array([[mu_o2Tpos,mu_o2Tpro,mu_o2Tneg],[mu_testTpos,mu_testTpro,0]])

lam = np.array([lam_o2,lam_test])

t_D = np.array([t_DTpos,t_DTpro,t_DTneg])

r = np.array([r_Tpos,r_Tpro,r_Tneg])

delta = np.array([delta_Tpos,delta_Tpro,delta_Tneg])

rho = np.array([rho_Tpos,rho_Tpro,rho_Tneg])

K_m = np.array([K_Tpos, K_Tpro, K_Tneg])

lim = np.array([[[l_lim_o2Tpos,u_lim_o2Tpos],[l_lim_o2Tpro,u_lim_o2Tpro],[l_lim_o2Tneg,u_lim_o2Tneg]],[[l_lim_testTpos,u_lim_testTpos],[l_lim_testTpro,u_lim_testTpro],[0,0]]],dtype=np.float64)

D = np.array([D_o2, D_test])

dataset = [t_max, dt, y0, p, mu, lam, r, K, delta, rho, K_m, lim, D, M]

In [11]:
def enveq(t, x , p , mu , lam , r , K , delta , rho , K_m, lim , D, M):
        
    # Seperating x into lists of variables, on length M
    o2 = []
    test = []
    Tpro = []
    Tpos = []
    Tneg = []
    for i in range(len(x)):
        if i % 5 == 0:
            Tpos.append(x[i])
                
        if i % 5 == 1:
            Tpro.append(x[i])
            
        if i % 5 == 2:
            Tneg.append(x[i])
                
        if i % 5 == 3:
            o2.append(x[i])
                
        if i % 5 == 4:
            test.append(x[i])
     
    do2 = []
    dtest = []
    dTpro = []
    dTpos = []
    dTneg = []
        
    for i in range(M):
        
        if i == 0:
            #Equation for oxygen: constant production, uptake by all 3 cells, decay
            d3 = p[0][i] - mu[0,0] * Tpos[i] - mu[0,1] * Tpro[i] - mu[0,2] * Tneg[i] - lam[0] * o2[i] + D[0] * (o2[i + 1] - 2 * o2[i])
            do2.append(d3)
            
            #Equation for testosterone: production by Tp, uptake by all Tp, T+, decay
            d4 = p[1] * Tpro[i] - mu[1,0] * Tpos[i] - mu[1,1] * Tpro[i] - lam[1] * test[i] + D[1] * (test[i + 1] - 2 * test[i])
            dtest.append(d4)
        
        if i == M-1:
            #Equation for oxygen: constant production, uptake by all 3 cells, decay
            d3 = p[0][i] - mu[0,0] * Tpos[i] - mu[0,1] * Tpro[i] - mu[0,2] * Tneg[i] - lam[0] * o2[i] + D[0] * (o2[i - 1] - 2 * o2[i]) 
            do2.append(d3)
            
            #Equation for testosterone: production by Tp, uptake by all Tp, T+, decay
            d4 = p[1] * Tpro[i] - mu[1,0] * Tpos[i] - mu[1,1] * Tpro[i] - lam[1] * test[i] + D[1] * (test[i - 1] - 2 * test[i])
            dtest.append(d4)
        elif 0 < i < M-1:
            #Equation for oxygen: constant production, uptake by all 3 cells, decay
            d3 = p[0][i] - mu[0,0] * Tpos[i] - mu[0,1] * Tpro[i] - mu[0,2] * Tneg[i] - lam[0] * o2[i] + D[0] * (o2[i - 1] + o2[i + 1] - 2 * o2[i])
            do2.append(d3)
            
            #Equation for testosterone: production by Tp, uptake by all Tp, T+, decay
            d4 = p[1] * Tpro[i] - mu[1,0] * Tpos[i] - mu[1,1] * Tpro[i] - lam[1] * test[i] + D[1] * (test[i - 1] + test[i + 1] - 2 * test[i])
            dtest.append(d4)
    
        #Equation for T+
        d0 = r[0] * Tpos[i] * (1 - ((Tpos[i] + Tpro[i] + Tneg[i])/(K + rho[0] * f_res(o2[i] , lim[0,0])*f_res(test[i],lim[1,0])))) - delta[0] * Tpos[i] 
        if Tpos[i] > K_m[0]: #If compartment exceeds maximum capacity for cells
            ovr = Tpos[i] % K_m[0]
            Tpos[i] = K_m[0]
            if i == 0: # If first compartment, sends cells to the 1st compartment
                Tpos[i + 1] += ovr  
                
            if i == M-1: #If last compartment, sends cells to M-1th compartment
                Tpos[i - 1] += ovr
                
            if 0 < i < M-1: #Otherwise equally divides overflow and sends to compartments before and after
                Tpos[i + 1] = 0.5*ovr
                Tpos[i - 1] = 0.5*ovr
           
        #Equation for Tp 
        d1 = r[1] * Tpro[i] * (1 - ((Tpos[i] + Tpro[i] + Tneg[i])/(K + rho[1] * f_res(o2[i], lim[0,1]) * f_res(test[i],lim[1,1])))) - delta[1] * Tpro[i]
        if Tpro[i] > K_m[1]: #If compartment exceeds maximum capacity for cells
            ovr = Tpro[i] % K_m[1]
            Tpro[i] = K_m[1]
            if i == 0: # If first compartment, sends cells to the 1st compartment
                Tpro[i + 1] += ovr  
                
            if i == M-1: #If last compartment, sends cells to M-1th compartment
                Tpro[i - 1] += ovr
               
            if 0 < i < M-1: #Otherwise equally divides overflow and sends to compartments before and after
                Tpro[i + 1] = 0.5*ovr
                Tpro[i - 1] = 0.5*ovr
                       
        #Equation for T-    
        d2 = r[2] * Tneg[i] * (1 - ((Tpos[i] + Tpro[i] + Tneg[i])/(K + rho[2] * f_res(o2[i],lim[0,2])))) - delta[2] * Tneg[i]
        if Tneg[i] > K_m[2]: #If compartment exceeds maximum capacity for cells
            ovr = Tneg[i] % K_m[2]
            Tneg[i] = K_m[2]
            if i == 0: # If first compartment, sends cells to the 1st compartment
                Tneg[i + 1] += ovr  
               
            if i == M-1: #If last compartment, sends cells to M-1th compartment
                Tneg[i - 1] += ovr
                
            if 0 < i < M-1: #Otherwise equally divides overflow and sends to compartments before and after
                Tneg[i + 1] = 0.5*ovr
                Tneg[i - 1] = 0.5*ovr                
        
        dTpro.append(d1)
        dTpos.append(d0)
        dTneg.append(d2)
     
    # Returns the list with dx_i_m/dt at that time
    dx = []
    for i in range(M):
        dx.append(dTpos[i])
        dx.append(dTpro[i])
        dx.append(dTneg[i])
        dx.append(do2[i])
        dx.append(dtest[i])

    return dx

#Resource Uptake function
def f_res(res, k):
    ll = k[0]
    ul = k[1]
    
    if res <= ll:
        return 0
    if res >= ul:
        return 1
    else:
        return (res - ll)/(ul - ll)    
    
    
    
    
### dataset = [t_max, dt, y0, p, mu, lam, r, K, delta, rho, lim, D, M]

#Function for solving system
def solve_eq(t_max, dt, y0, p, mu, lam, r, K, delta, rho, K_m, lim, D, M, f_name):
    
    #Timeseries arrays
    t0 = 0
    t = [t0]
    y_0 = np.reshape(y0, (M, 5))
    y = [y_0]
    y_t = y0
    
    #ODE declaration
    #f_ode = ode(enveq).set_integrator('lsoda').set_initial_value(y0).set_f_params(p,mu,lam,r,K,delta,rho,lim, M)
    
    f_ode = ode(lambda t,y: enveq(t , y , p , mu , lam , r , K , delta , rho , K_m, lim , D, M)).set_integrator('lsoda')
    f_ode.set_initial_value(y0, t0)    
    while f_ode.t < t_max:
        t.append(f_ode.t+dt)
        y_t = f_ode.integrate(f_ode.t+dt)
                      
        for i in range(len(y_t)):
            if y_t[i] < 0: # If any value goes negative
                y_t[i] = 0 # Sets neg values to 0
                f_ode.set_initial_value(y_t,f_ode.t) #if anomaly detected, reset initial conditions to zero set values at that time t

        y_t = np.reshape(y_t, (M, 5))
        
        for i in range(len(y_t)):
            if np.logical_and(y_t[i][0:3]>0 , y_t[i][0:3]<1).any(): # If cell numbers drop below 1
                y_t[i][0:3] = np.where(y_t[i][0:3] >= 1, y_t[i][0:3], np.zeros(np.shape(y_t[i][0:3]))) #Set cell number to 0
                

        y.append(y_t)
        
        if not f_ode.successful(): #Check if integration fails
            print('Failure @',f_ode.t)
                
    result= [t, y]
### returns time-series data as dataframes of values
#M - nuumber of compartments, N - number of variables
# n = 0 - Tpos, 1 - Tpro, 2 - Tneg, 3 - O2, 4 - test
# m = Compartment number
    
def cleaner(result, M, f_name):
    tk = []
    for i in result[0]:
        for j in range(M):
            tk.append(i)
            
    t_steps = [i/(24 * 60) for i in tk]  # Time steps in days
    data = {"Time": t_steps}    
    pop = result[1]
 
     # Returns time-series data as dataframes of values    
    tpos = []
    tpro = []
    tneg = []
    o2 = []
    test = []
    comps = []
    for i in range(len(pop)):
        for m in range(M):
            tpos.append(pop[i][m, 0])
            tpro.append(pop[i][m, 1])
            tneg.append(pop[i][m, 2])
            o2.append(pop[i][m, 3])
            test.append(pop[i][m, 4])
            comps.append(m)
    
    dict0 = {"Tpos" : tpos, "Tpro" : tpro, "Tneg" : tneg, "O2" : o2, "Test" : test, "Compartment" : comps}
    data.update(dict0)
   
    cell_tot = []
    for i in range(len(tpos)):
        cell_tot.append(tpos[i] + tpro[i] + tneg[i]) # Total Cell count per compartment
    dict1 = {"Total Cells" : cell_tot}
    data.update(dict1)
                           
    tot_cell = []
    for i in range(len(pop)): # Counting total cell population across all compartments
        k = 0
        for m in range(M):
            k += pop[i][m, 0] + pop[i][m, 1] + pop[i][m, 2]
            
        tot_cell.append(k)
    t_burd = []
    for i in tot_cell:
        for m in range(M):
            t_burd.append(i)
                        
    dict3 = {"Tumour Burden" : t_burd}
    data.update(dict3)
    df = pd.DataFrame(data)        
    df.to_csv(r'C:\Users\Aditya\Documents\Python\Cancer Model\Cancer-Comp-Strat-Spatial\raw-output\_' + f_name + '.csv',index=False)  
    return df

In [12]:
### Calculating Solution of System

data = solve_eq(t_max, dt, y0, p, mu, lam, r, K, delta, rho, K_m, lim, D, M, "well-mixed")

In [13]:
sns.relplot(x = "Time", y = "Total Cells", kind="line", data = data)

ValueError: Could not interpret input 'Time'

In [ ]:
print(data)